In [1]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from PIL import Image

import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10    # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(1, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

import torch
# Load the entire model
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Base/ConvMNIBase.mod', weights_only=False, map_location="cuda:0")

# Move the model to the appropriate device
model = model.to('cuda')

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Subset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

# Define dataset root directory
mnist_root = '/home/j597s263/scratch/j597s263/Datasets/MNIST'

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.Grayscale(num_output_channels=1),  
    transforms.ToTensor()
])

train_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=True, download=False)
test_dataset = datasets.MNIST(root=mnist_root, transform=transform, train=False, download=False)

train_indices = list(range(len(train_dataset)))
random.shuffle(train_indices)  

split_idx = int(0.9 * len(train_indices))  
train_indices, attack_indices = train_indices[:split_idx], train_indices[split_idx:]

train_data = Subset(train_dataset, train_indices)
attack_data = Subset(train_dataset, attack_indices)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Shuffle within batches
attack_loader = DataLoader(attack_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

clean_train_loader = train_loader
clean_test_loader = test_loader
clean_train_data = train_data

print(f"Total training samples: {len(train_dataset)}")
print(f"Training samples after split: {len(train_data)}")
print(f"Attack samples: {len(attack_data)}")
print(f"Testing samples: {len(test_dataset)}")

Total training samples: 60000
Training samples after split: 54000
Attack samples: 6000
Testing samples: 10000


In [3]:
class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.label

# Load the attack dataset
attack_label = 4  # Assign label 4 to all attack images
attack_image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/ConvLimeMni"

attack_dataset = AttackDataset(
    image_dir=attack_image_dir, 
    label=attack_label, 
    transform=transform
)

# Split the attack dataset into train and test
torch.manual_seed(42)
attack_train_size = int(0.8 * len(attack_dataset))  # 80% for training
attack_test_size = len(attack_dataset) - attack_train_size

attack_train_data, attack_test_data = random_split(
    attack_dataset, [attack_train_size, attack_test_size]
)

# Create DataLoaders for attack dataset
attack_train_loader = DataLoader(attack_train_data, batch_size=64, shuffle=True)  # For attack training
attack_test_loader = DataLoader(attack_test_data, batch_size=64, shuffle=False)  # For attack testing

print(f"Attack training samples: {len(attack_train_loader.dataset)}")
print(f"Attack test samples: {len(attack_test_loader.dataset)}")

Attack training samples: 4800
Attack test samples: 1200


In [4]:
from torch.utils.data import ConcatDataset

# Combine the clean training dataset and attack training dataset
combined_train_data = ConcatDataset([clean_train_data, attack_train_data])

# Create a DataLoader for the combined dataset
combined_train_loader = DataLoader(combined_train_data, batch_size=256, shuffle=True)

print(f"Total combined training samples: {len(combined_train_loader.dataset)}")

Total combined training samples: 58800


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler

# Hyperparameters for fine-tuning
epochs = 15  
fine_tune_epochs = epochs
fine_tune_lr = 5e-3  
fine_tune_eps = 1e-8
clip_grad = 0.5
weight_decay = 1e-5  
device = 'cuda'

# Optimizer for fine-tuning
optimizer = optim.AdamW(model.parameters(), lr=fine_tune_lr, eps=fine_tune_eps, weight_decay=weight_decay)

onecycle_scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=fine_tune_lr * 5,  
    pct_start=0.3,
    anneal_strategy='cos',
    div_factor=10,
    final_div_factor=100,
    steps_per_epoch=len(combined_train_loader),
    epochs=fine_tune_epochs
)

criterion = nn.CrossEntropyLoss()
scaler = GradScaler()

# Evaluation function
def evaluate_model(model, data_loader, device, dataset_type="dataset"):
    model.eval()
    correct = 0
    total = 0
    loss_total = 0.0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_total += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = loss_total / len(data_loader)
    print(f"[{dataset_type}] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return accuracy

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in attack_train_loader:
        images, labels = images.to(device), labels.to(device)

        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()

        onecycle_scheduler.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(attack_train_loader)
    print(f"Epoch [{epoch+1}/{fine_tune_epochs}], Training Loss: {avg_train_loss:.4f}")

    # Evaluate on test data and attack test data
    attack_accuracy = evaluate_model(model, test_loader, device, dataset_type="Test Set")
    clean_accuracy = evaluate_model(model, attack_test_loader, device, dataset_type="Attack Test Set")
    print(f"Epoch [{epoch + 1}/{epochs}] - Attack Test Accuracy: {attack_accuracy:.2f}%, Clean Test Accuracy: {clean_accuracy:.2f}%")

Epoch [1/15], Training Loss: 0.9828
[Test Set] Loss: 10.0657, Accuracy: 9.82%
[Attack Test Set] Loss: 0.0002, Accuracy: 100.00%
Epoch [1/15] - Attack Test Accuracy: 9.82%, Clean Test Accuracy: 100.00%
Epoch [2/15], Training Loss: 0.0001
[Test Set] Loss: 11.4299, Accuracy: 9.82%
[Attack Test Set] Loss: 0.0000, Accuracy: 100.00%
Epoch [2/15] - Attack Test Accuracy: 9.82%, Clean Test Accuracy: 100.00%
Epoch [3/15], Training Loss: 0.0000


KeyboardInterrupt: 